In [1]:
!pip install -q transformers sentence-transformers
!pip install -q mteb wandb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 29.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.4/287.4 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.6/177.6 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.6.1 which is incompatible.


In [2]:
from torch.utils.data import DataLoader
from sentence_transformers import SentenceTransformer, losses, models, util
from sentence_transformers.readers import InputExample
import os
import pandas as pd
from ast import literal_eval

## 1. Download data

In [3]:
# from google.colab import drive
# import os
# import shutil
# from google.colab import auth
# import gdown

# def download_drive_folder(folder_id, destination_path):
#     os.makedirs(destination_path, exist_ok=True)

#     folder_url = f"https://drive.google.com/drive/folders/{folder_id}"
#     !gdown --folder {folder_url} -O {destination_path}

# folder_id = "1HyF8-EfL4w0G3spBbhcc0jTOqdc4XUhB"
# download_drive_folder(folder_id, "/content/downloaded_folder")

In [4]:
!wget https://huggingface.co/datasets/tmnam20/BKAI-Legal-Retrieval/resolve/main/archive.zip
!unzip -o -qq archive.zip -d data

--2024-12-26 06:00:32--  https://huggingface.co/datasets/tmnam20/BKAI-Legal-Retrieval/resolve/main/archive.zip
Resolving huggingface.co (huggingface.co)... 13.33.45.10, 13.33.45.37, 13.33.45.68, ...
Connecting to huggingface.co (huggingface.co)|13.33.45.10|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.hf.co/repos/77/34/7734ba2101c276edd0fc8c24cc368330a94a50e26dadb6b459834f0603785f20/35bef18231742a2ec66889a9074a8283b0807448b777bf8dda45a95859d0e875?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27archive.zip%3B+filename%3D%22archive.zip%22%3B&response-content-type=application%2Fzip&Expires=1735452032&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTczNTQ1MjAzMn19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmhmLmNvL3JlcG9zLzc3LzM0Lzc3MzRiYTIxMDFjMjc2ZWRkMGZjOGMyNGNjMzY4MzMwYTk0YTUwZTI2ZGFkYjZiNDU5ODM0ZjA2MDM3ODVmMjAvMzViZWYxODIzMTc0MmEyZWM2Njg4OWE5MDc0YTgyODNiMDgwNzQ0OGI3NzdiZjhkZGE0

### 2. Preprocessing data

In [5]:
corpus_data = pd.read_csv("/content/data/corpus.csv")
train_data = pd.read_csv("/content/data/train_split.csv", converters={"context": literal_eval})
val_data = pd.read_csv("/content/data/val_split.csv", converters={"context": literal_eval})


In [6]:
corpus_data.head()

,text,cid
0,"Thông tư này hướng dẫn tuần tra, canh gác bảo ...",0
1,"1. Hàng năm trước mùa mưa, lũ, Ủy ban nhân dân...",1
2,Tiêu chuẩn của các thành viên thuộc lực lượng ...,2
3,"Nhiệm vụ của lực lượng tuần tra, canh gác đê\n...",3
4,"Phù hiệu của lực lượng tuần tra, canh gác đê\n...",4


In [7]:
print("len(train_data): ", len(train_data))
train_data.head()

len(train_data):  89592


,question,context,cid,qid
0,Liên đoàn Luật sư Việt Nam là tổ chức xã hội –...,[“Điều 2. Địa vị pháp lý của Liên đoàn Luật sư...,[142820],72600
1,Tên hợp tác xã bị rơi vào trường hợp cấm thì c...,"[Tên hợp tác xã, liên hiệp hợp tác xã\n1. Tên ...",[27817 72117],147562
2,Tài xế lái xe ô tô khách 50 chỗ ngồi bao lâu t...,"[""1. Sử dụng lái xe bảo đảm sức khỏe theo tiêu...",[33215 56201],142107
3,Các bước chuẩn bị thủ thuật bó bột Cravate sẽ ...,[BỘT CRAVATE\n...\nIV. CHUẨN BỊ\n1. Người thực...,[148158],77353
4,Viên chức Hộ sinh hạng 4 có những nhiệm vụ gì ...,[Hộ sinh hạng IV - Mã số: V.08.06.16\n1. Nhiệm...,[188132],113090


In [8]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 89592 entries, 0 to 89591
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   question  89592 non-null  object
 1   context   89592 non-null  object
 2   cid       89592 non-null  object
 3   qid       89592 non-null  int64 
dtypes: int64(1), object(3)
memory usage: 2.7+ MB


In [9]:
print("len(val_data): ", len(val_data))
val_data.head()

len(val_data):  29864


,question,context,cid,qid
0,Phó Tổng Giám đốc Ngân hàng Chính sách xã hội ...,[Áp dụng chế độ tiền lương và phụ cấp quy định...,[140864],70867
1,Ai có thẩm quyền quyết định thành lập Hội đồng...,[Thành lập Hội đồng\n1. Bộ trưởng Bộ Y tế ra q...,[62339],813
2,Thời hiệu xử phạt đối với nhà xuất bản thực hi...,[Mức phạt tiền và thẩm quyền xử phạt\n...\n4. ...,[63171],40392
3,"Việc ký kết, thực hiện thỏa thuận quốc tế nhân...",[Báo cáo tình hình ký kết và thực hiện thỏa th...,[157761],85946
4,Đề án sử dụng tài sản công tại đơn vị sự nghiệ...,"[""Điều 44. Đề án sử dụng tài sản công tại đơn ...",[95397],55607


In [10]:
val_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29864 entries, 0 to 29863
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   question  29864 non-null  object
 1   context   29864 non-null  object
 2   cid       29864 non-null  object
 3   qid       29864 non-null  int64 
dtypes: int64(1), object(3)
memory usage: 933.4+ KB


In [11]:
# chuyển cid về dạng array of integer

train_data["cid"] = train_data["cid"].apply(lambda x: [int(i) for i in x[1: -1].split()])
val_data["cid"] = val_data["cid"].apply(lambda x: [int(i) for i in x[1: -1].split()])

In [12]:
# chuẩn bị dữ liệu với mỗi samle là cặp query và context tương ứng


samples = {"train": [], "test": []}
data = {"train": train_data, "test": val_data}


for subset in ["train", "test"]:
    for i, row in data[subset].iterrows():
        context = row['context']
        question = row['question']
        for c in context:
            samples[subset].append(InputExample(texts=[question, c]))

train_dataloader = DataLoader(samples["train"], batch_size=32, shuffle=True)

### 3. Load pretrained model

In [13]:
model_id = "google-bert/bert-base-multilingual-cased"
embed_model = models.Transformer(model_id,
                                 max_seq_length=512,
                                 cache_dir="./cached")

# tổng hợp các embedding từ được tạo bởi BERT thành embedding câu duy nhất.
pooling_model = models.Pooling(
    embed_model.get_word_embedding_dimension(), # lấy chiều embedding từ BERT
    pooling_mode_mean_tokens=False, # embedding câu = trung bình cộng của embedding từ
    pooling_mode_cls_token=True, # token phân loại câu không được sử dụng
    pooling_mode_max_tokens=False,
)


model = SentenceTransformer(modules=[embed_model, pooling_model],
                            device="cuda",
                            cache_folder ="./cached")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

### 4. Train the embedding model

Để train mô hình embedding với các cặp (query, context), ta sẽ sử dụng hàm ```Multiple Negatives Ranking Loss.```
Tuy nhiên, do hạn chế về mặt phần cứng, chúng ta sẽ dùng phiên bản Cached ```Multiple Negatives Ranking Loss```
để có thể huấn luyện mô hình với batch size lớn trên GPU T4 16GB của colab


In [ ]:
NUM_EPOCHS = 2
MODEL_SAVE_PATH = "./output"
train_loss = losses.CachedMultipleNegativesRankingLoss(model=model)

model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs = NUM_EPOCHS,
    steps_per_epoch=NUM_EPOCHS,
    output_path=MODEL_SAVE_PATH,
    optimizer_params = {"lr": 3e-5},
    show_progress_bar=True
)

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss
500,1.787400
1000,0.189700
1500,0.130000
2000,0.102000
2500,0.087400
3000,0.076900
3500,0.059900


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

model.save_to_hub("bert-legal-document-retrieval")

### 5. Evaluate model

In [ ]:
from mteb import MTEB
from mteb.abstasks.AbsTaskRetrieval import AbsTaskRetrieval
from mteb.abstasks.TaskMetadata import TaskMetadata

In [ ]:
class BKAILegalDocRetrievalTask(AbsTaskRetrieval):
    metadata = TaskMetadata(
        name="BKAILegalDocRetrieval",
        description="",
        reference="https://github.com/embeddings-benchmark/mteb/blob/main/docs/adding_a_dataset.md",
        type="Retrieval",
        category="c2p",
        modalities=["text"],
        eval_splits=["test"],
        eval_langs=["vi"],
        main_score="ndcg_at_10",
        dataset={
            "path": "data",
            "revision": "d4c5a8ba10ae71224752c727094ac4c46947fa29",
        },
        date=("2012-01-01", "2020-01-01"),
        form="Written",
        domains=["Academic", "Non-fiction"],
        task_subtypes=["Scientific Reranking"],
        license="cc-by-nc-4.0",
        annotations_creators="derived",
        dialects=[],
        text_creation="found",
        bibtex_citation="",
    )

    data_loaded = True

    def __init__(self, **krawgs):
        super().__init__(**krawgs)

        global corpus_data, data

        self.corpus = {}
        self.queries = {}
        self.relevant_docs = {}

        shared_corpus = {}
        for _, row in corpus_data.iterrows():
            cid_str = f"c{row['cid']}"
            shared_corpus[cid_str] = {"text": row["text"], "_id": row["cid"]}

        for subset in data.keys():
            self.corpus[subset] = shared_corpus
            self.queries[subset] = {}
            self.relevant_docs[subset] = {}

        for subset in data.keys():
            for i, row in data[subset].iterrows():
                question = row['question']
                cid = row['cid']
                qid = row['qid']
                qid_str = f"q{qid}"
                cids_str = [f"c{c}" for c in cid]

                self.queries[subset][qid_str] = question
                for cid_str in cids_str:
                    if cid not in self.relevant_docs[subset]:
                        self.relevant_docs[subset][qid_str] = {}
                    self.relevant_docs[subset][qid_str][cid_str] = 1
        self.data_loaded = True


custom_task = BKAILegalDocRetrievalTask()
evaluation = MTEB(custom_task)
evaluation.run(model)

### 6. Query

In [ ]:
passages = corpus_data["text"].tolist()
print(f"Num of passages: {len(passages)}")

corpus_embeddings = model.encode(passages, convert_to_tensor=True, show_progress_bar=True)


def search(query: str):
    print(f"Query: {query}")
    question_embedding = model.encode(query, convert_to_tensor=True)
    question_embedding = question_embedding.cuda()
    hits = util.semantic_search(question_embedding, corpus_embeddings, top_k=10)
    hits = hits[0]

    print("\n--------------------------------\n")
    print("Top-3 Bi-Encoder retrieval hits")
    hits = sorted(hits, key=lambda x: x['score'], reverse=True)
    for hit in hits[0:3]:
        print("\t{:.3f}\t{}".format(hit['score'], passages[hit['corpus_id']].replace("\n", " ")))

    return hits


search("Lực lượng tuần tra, canh gác đê được trang bị những gì?")